In [1]:
# -------------------------------------------------------------------------------------------------------------------
# Course name - Exploratory data analysis
# Course code - DSC530
# Week 9 assignment
# Script name - Shekhar530Week9.ipynb
# Creator - Manish Shekhar
# Date created - Feb 6th - Feb 12th 2023
# -------------------------------------------------------------------------------------------------------------------

####  Example 11-1 : 
####  Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming
####  that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to
####  variables that are known before the birth, and likely to be available to the people in the pool.

In [59]:
# Get the required functions
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [60]:
# import required libraries
import numpy as np

import random

import thinkstats2
import thinkplot
import statsmodels.formula.api as smf

In [61]:
# Hypothesis test with essential methods 
class HypothesisTest(object):

    def __init__(self, data):
        self.data = data
        self.MakeModel()
        self.actual = self.TestStatistic(data)

    def PValue(self, iters=1000):
        self.test_stats = [self.TestStatistic(self.RunModel()) 
                           for _ in range(iters)]

        count = sum(1 for x in self.test_stats if x >= self.actual)
        return count / iters

    def TestStatistic(self, data):
        raise UnimplementedMethodException()

    def MakeModel(self):
        pass

    def RunModel(self):
        raise UnimplementedMethodException()

In [62]:
# download required data files 
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

In [63]:
# define live, first, and others datasets
import first

live, firsts, others = first.MakeFrames()
data = firsts.prglngth.values, others.prglngth.values
live = live[live.prglngth>30]

In [64]:
# And we can search for variables with explanatory power.
# Because we don't clean most of the variables, we are probably missing some good ones.

import patsy

def GoMining(df):
    """Searches for variables that predict birth sex.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ agepreg + ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared, name))

    return variables

In [65]:
variables = GoMining(live)
variables

[(5.4611774921276e-05, 'caseid'),
 (0.0006488168245175618, 'pregordr'),
 (0.002249407606621401, 'pregend1'),
 (0.0045801411639411205, 'nbrnaliv'),
 (0.0004105037064817685, 'cmprgend'),
 (2.481706396606498e-05, 'cmprgbeg'),
 (0.0016590163592549168, 'gestasun_m'),
 (0.0010938126121889935, 'gestasun_w'),
 (0.8065115675039275, 'wksgest'),
 (0.09562434659774688, 'mosgest'),
 (1.8878682399137148e-05, 'bpa_bdscheck1'),
 (0.0002115324987046474, 'babysex'),
 (0.12067945355791987, 'birthwgt_lb'),
 (0.00025008146724392333, 'birthwgt_oz'),
 (0.0004105037064817685, 'cmbabdob'),
 (2.0901979782106395e-05, 'kidage'),
 (3.025183042248969e-05, 'hpagelb'),
 (0.0014368115355251065, 'matchfound'),
 (0.002918483981364406, 'anynurse'),
 (0.0005402902882049032, 'frsteatd_n'),
 (0.0013304832813814116, 'frsteatd_p'),
 (0.0002130347575965974, 'frsteatd'),
 (0.002047727311564329, 'cmlastlb'),
 (0.00017932809415743822, 'cmfstprg'),
 (0.0012940540023292924, 'cmlstprg'),
 (0.0008906016746064171, 'cmintstr'),
 (0.000

In [66]:
# The following functions report the variables with the highest values of  𝑅2 .
import re

def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

In [67]:
import pandas as pd

# Some of the variables that do well are not useful for prediction because they are not known ahead of time.
MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8065115675039275 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
agecon 0.7772278070905596 AGE AT TIME OF CONCEPTION
totalwgt_lb 0.12550330803309873
birthwgt_lb 0.12067945355791987 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10383887474318443 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562434659774688 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.022418420753651858 PRGLNGTH IMPUTATION FLAG
nbrnaliv 0.0045801411639411205 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
anynurse 0.002918483981364406 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.0025670121587798578 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.002249407606621401 BC-1 HOW PREGNANCY ENDED - 1ST MENTION
cmlastlb 0.002047727311564329 CM FOR R'S MOST RECENT LIVE BIRTH
fmarcon5_i 0.001971805916452962 FMARCON5 IMPUTATION FLAG
evuseint 0.0019247289345255547 EG-1 USE ANY METHOD IN PREGNANCY INTER

/var/folders/tb/46g9m78d4y1b91l8ybsbqc980000gq/T/ipykernel_29443/1039546438.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


In [68]:
# The following are the only variables I found that have a statistically significant effect on pregnancy length
# and were known by the 30th week of pregnancy. Considering only variables known by 30th week of pregnancy.

# agecon 0.7772278070905596 AGE AT TIME OF CONCEPTION
# cmlastlb 0.002047727311564329 CM FOR R'S MOST RECENT LIVE BIRTH
# poverty 0.0013131375600279327 POVERTY LEVEL INCOME
# birthord 0.0012989497516830983 BIRTH ORDER
# educat 0.0008059836909062312 EDUCATION (COMPLETED YEARS OF SCHOOLING)
# fmarital 0.0007140103748330962 FORMAL MARITAL STATUS
# hispanic 0.0007028741902321833 HISPANIC ORIGIN

import statsmodels.formula.api as smf
# model = smf.ols('prglngth ~ agecon + cmlastlb + poverty + birthord + educat + fmarital + C(race)', data=live)
# R-squared:	0.005
# Adj. R-squared:	0.004
# F-statistic:	5.905
# coef	std err	t	P>|t|	[0.025	0.975]
# Intercept	39.1745	0.161	243.656	0.000	38.859	39.490
# agecon	-5.315e-05	4.52e-05	-1.176	0.240	-0.000	3.54e-05
# cmlastlb	-0.0003	6.08e-05	-4.121	0.000	-0.000	-0.000
# poverty	0.0002	0.000	1.394	0.163	-9.54e-05	0.001
# birthord	-0.0424	0.022	-1.907	0.057	-0.086	0.001
# educat	0.0055	0.010	0.555	0.579	-0.014	0.025
# fmarital	-0.0204	0.013	-1.548	0.122	-0.046	0.005
# hispanic	0.0769	0.048	1.587	0.112	-0.018	0.172

# model = smf.ols('prglngth ~ agecon + cmlastlb + birthord==1 + race==2', data=live)
#	coef	std err	t	P>|t|	[0.025	0.975]
#Intercept	39.0736	0.123	318.677	0.000	38.833	39.314
#birthord == 1[T.True]	0.1123	0.043	2.630	0.009	0.029	0.196
#race == 2[T.True]	0.1414	0.043	3.296	0.001	0.057	0.225
#agecon	-1.649e-05	3.89e-05	-0.424	0.671	-9.27e-05	5.97e-05
#cmlastlb	-0.0003	6.08e-05	-4.246	0.000	-0.000	-0.000
# agecon has p-value 0.671 which mean that there are 67% chance for null hyoothesis to be true

# model = smf.ols('prglngth ~ cmlastlb + birthord==1 + race==2 + C(nbrnaliv)', data=live)
# 	coef	std err	t	P>|t|	[0.025	0.975]
#Intercept	39.0666	0.080	486.348	0.000	38.909	39.224
#birthord == 1[T.True]	0.1022	0.040	2.549	0.011	0.024	0.181
#race == 2[T.True]	0.1418	0.042	3.381	0.001	0.060	0.224
#C(nbrnaliv)[T.2.0]	-1.7652	0.179	-9.840	0.000	-2.117	-1.414
#C(nbrnaliv)[T.3.0]	-0.3099	0.523	-0.592	0.554	-1.336	0.716
#C(nbrnaliv)[T.4.0]	0.0567	0.943	0.060	0.952	-1.791	1.905
#C(nbrnaliv)[T.5.0]	0.1459	0.843	0.173	0.863	-1.507	1.799
#cmlastlb	-0.0003	6.04e-05	-4.351	0.000	-0.000	-0.000
# Only nbrliv==2 is statistically significant and we can reject the null hypothesis saying it does not have any effect on the dependent variable.

# model = smf.ols('prglngth ~ cmlastlb + birthord==1 + race==2 + nbrnaliv==2', data=live)
# R^2 = 0.015 for this model is best so far 
# Adjusted R-squared is 0.015
# F-statistic - 33.91
# The condition number is large, 1.09e+04. This might indicate that there are strong multicollinearity or other numerical problems.

model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
# R^2 is 0.011
# Adjusted R-squared is 0.011
# F-statistic - 34.28

results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Fri, 10 Feb 2023   Prob (F-statistic):           5.09e-22
Time:                        23:37:47   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000      38.686      38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011       0.023       0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001       0.057       0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Exercise 11-3: If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called poisson. It works the same way as ols and logit. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called numbabes.

#### Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [69]:
# combine variables from pregnancy and respondant data 
import nsfg

resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')
join.shape

(8884, 3331)

In [70]:
# Preparing data to use non linear model of age
join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2
# join['age3'] = join.age_r**3


/var/folders/tb/46g9m78d4y1b91l8ybsbqc980000gq/T/ipykernel_29443/2165479471.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  join['age2'] = join.age_r**2


In [71]:
# craeting the Poisson regression model and fitting the data
# formula = 'numbabes ~ age_r + age2 + age3 + C(race) + totincr + educat'
formula = 'numbabes ~ age_r + age2 + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.677002
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8877
Method:                           MLE   Df Model:                            6
Date:                Fri, 10 Feb 2023   Pseudo R-squ.:                 0.03686
Time:                        23:38:12   Log-Likelihood:                -14898.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                3.681e-243
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.0324      0.169     -6.098      0.000      -1.364      -0.701
C(race)[T.2]    -0.1401      0.015     -9.479      0.000      -0.169      -0.111
C(race)[T.3]    -0.0991      0.025     -4.029      0.000      -0.147      -0.051
age_r            0.1556      0.010     15.006      0.000       0.135       0.176
age2            -0.0020      0.000    -13.102      0.000      -0.002      -0.002
totincr         -0.0187      0.002     -9.830      0.000      -0.022      -0.015
educat          -0.0471      0.003    -16.076      0.000      -0.053      -0.041
================================================================================
"""

In [72]:
# Now we can predict the number of children for a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000
columns = ['age_r', 'age2', 'race', 'totincr', 'educat']

# join.race.value_counts()
# race 1 for black
# join.totincr.value_counts()
# 14 for total income
# join.educat.value_counts()
# 16 for college graduate

new = pd.DataFrame([[35, 35**2, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.496802
dtype: float64

#### Exercise 11-4: If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels
#### with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never #### married; in the NSFG dataset, marital status is encoded in a variable called rmarital. Suppose you meet a woman who is 25 years old, white, and a high
#### school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [73]:
# Build model and fit the data 
formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 1.084053
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8849
Method:                           MLE   Df Model:                           30
Date:                Fri, 10 Feb 2023   Pseudo R-squ.:                  0.1682
Time:                        23:38:30   Log-Likelihood:                -9630.7
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.0156      0.805     11.199      0.000       7.438      10.593
C(race)[T.2]    -0.9237      0.089    -10.418      0.000      -1.097      -0.750
C(race)[T.3]    -0.6179      0.136     -4.536      0.000      -0.885      -0.351
age_r           -0.3635      0.051     -7.150      0.000      -0.463      -0.264
age2             0.0048      0.001      6.103      0.000       0.003       0.006
totincr         -0.1310      0.012    -11.337      0.000      -0.154      -0.108
educat          -0.1953      0.019    -10.424      0.000      -0.232      -0.159
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.9570      3.020      0.979      0.328      -2.963       8.877
C(race)[T.2]    -0.4411      0.237     -1.863      0.062      -0.905       0.023
C(race)[T.3]     0.0591      0.336      0.176      0.860      -0.600       0.718
age_r           -0.3177      0.177     -1.798      0.072      -0.664       0.029
age2             0.0064      0.003      2.528      0.011       0.001       0.011
totincr         -0.3258      0.032    -10.175      0.000      -0.389      -0.263
educat          -0.0991      0.048     -2.050      0.040      -0.194      -0.004
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.5238      1.205     -2.924      0.003      -5.886      -1.162
C(race)[T.2]    -0.3213      0.093     -3.445      0.001      -0.504      -0.139
C(race)[T.3]    -0.7706      0.171     -4.509      0.000      -1.106      -0.436
age_r            0.1155      0.071      1.626      0.104      -0.024       0.255
age2            -0.0007      0.001     -0.701      0.483      -0.003       0.001
totincr         -0.2276      0.012    -19.621      0.000      -0.250      -0.205
educat           0.0667      0.017      3.995      0.000       0.034       0.099
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8963      1.305     -2.220      0.026      -5.453      -0.339
C(race)[T.2]    -1.0407      0.104    -10.038      0.000      -1.244      -0.837
C(race)[T.3]    -0.5661      0.156     -3.635      0.000      -0.871      -0.261
age_r            0.2411      0.079      3.038      0.002       0.086       0.397
age2            -0.0035      0.001     -2.977      0.003      -0.006      -0.001
totincr         -0.2932      0.015    -20.159      0.000      -0.322      -0.265
educat          -0.0174      0.021     -0.813      0.416      -0.059       0.025
-

In [74]:
# Make a prediction for a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000.
# This person has a 75% chance of being currently married, 
# a 13% chance of being "not married but living with opposite 
# sex partner", etc.

columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.750028,0.126397,0.001564,0.033403,0.021485,0.067122
